# 웹페이지 크롤링 이후 RAG로 활용해보기

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

In [6]:
!pip install -U langchain-community unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=c0f003a0df0bc9184eeccb7ae5cdccb0b63a7294acb703bbc64fbbb63acd4ddc
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = 'my_api_key'

In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [16]:
from bs4 import BeautifulSoup
import requests

# 1. 웹페이지 크롤링 및 텍스트 추출
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    )
}
url = "https://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')  # 모든 <p> 태그에서 텍스트 추출
    content = "\n".join([para.get_text(strip=True) for para in paragraphs])
else:
    raise Exception(f"Failed to fetch the webpage. Status code: {response.status_code}")


# 2. LangChain 문서 객체 생성
documents = [Document(page_content=content, metadata={"source": url})]

In [17]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [18]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len
)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

<ipython-input-18-5d342ab71e02>:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tok

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [20]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

openai = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18",
                    streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
                    temperature = 0)

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k':3, 'fetch_k': 10}),
                                 return_source_documents = True)

query = "지구는 노아의 홍수 전체 기간동안 물로 뒤덮였나요? 주어진 문서를 기반으로 답변하세요"
result = qa(query)

주어진 문서에 따르면, 노아의 대홍수로 육지가 물에 잠긴 기간은 6-7개월 정도로 짧아질 수 있다고 언급되어 있습니다. 따라서 지구가 노아의 홍수 전체 기간 동안 물로 뒤덮였다고 단정할 수는 없습니다.

In [21]:
result

{'query': '지구는 노아의 홍수 전체 기간동안 물로 뒤덮였나요? 주어진 문서를 기반으로 답변하세요',
 'result': '주어진 문서에 따르면, 노아의 대홍수로 육지가 물에 잠긴 기간은 6-7개월 정도로 짧아질 수 있다고 언급되어 있습니다. 따라서 지구가 노아의 홍수 전체 기간 동안 물로 뒤덮였다고 단정할 수는 없습니다.',
 'source_documents': [Document(metadata={'source': 'https://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view'}, page_content='기본적 논리\n식물이나 씨앗이 어떻게 1년 동안 물속에서 생존할 수 있었을까? 노아 방주에서 내린 후에 초식동물들은 무엇을 먹었을까? 이런 종류의 질문들은 흔히 성경 비판론자와 기독교인들로부터 동일하게 받는 질문이다. 불행히도, 이것은 때때로 믿는 자들에게도 하나님의 말씀을 의심하고 인간의 이론을 수용하게 되는 원인이 된다. 아래의 글에서 볼 수 있는 것처럼, 침수된 상태에서 어떤 식물이나 씨앗이 살아남는 방법은 간단하고 무수히 많다. 홍수의 진행 단계에 대하여 성경이 실제로 우리에게 알려주는 것에 관한 몇몇 오해들을 주목하는 것 역시 가치 있는 일이다.'),
  Document(metadata={'source': 'https://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view'}, page_content='땅은 감람나무가 자라서 잎사귀를 낼 만큼 충분한 기간 동안 있었음을 보여준다. 따라서 노아의 대홍수로 육지가 물에 잠긴 기간은 6-7개월 정도로 짧아질 수도 있다.'),
  Document(metadata={'source': 'https://creation.kr/EvidenceofFlood/?idx=1288454&bmode=view'}, page_content='그림 1. 홍수의 시간표')]}